### create resources

In [1]:
import time

from azure.identity import AzureCliCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.resource import ResourceManagementClient

import tiktoken 

In [2]:
subscription_id  = '<your-subscription-id>'
region = 'eastus'

project = 'testopenai'
region = 'eastus'

environment = '-dev-'
resource_group_name = 'rg-' + project + environment + region + '-001' # create names using convention
openai_name = 'oai-' + project + environment + region + '-001'
deployment_name = 'gpt-' + project + environment + region + '-001'

In [3]:
credential = AzureCliCredential()
resource_client = ResourceManagementClient(credential, subscription_id) # instanciate client object
cognitive_client = CognitiveServicesManagementClient(credential=AzureCliCredential(), subscription_id=subscription_id)

In [4]:
cognitive_account = cognitive_client.accounts.get(resource_group_name = resource_group_name,
                                                  account_name = openai_name)

### Call API

In [5]:
import openai

In [6]:
key = cognitive_client.accounts.list_keys(resource_group_name = resource_group_name,
                                                                 account_name = openai_name).key1
endpoint = cognitive_account.properties.endpoint

In [7]:
openai.api_type = "azure"
openai.api_base = endpoint
openai.api_version = "2022-12-01"
openai.api_key = key

In [8]:
class Encoder():
    def __init__(self, 
                 name: str, 
                 base: str = 'cl100k_base', 
                 start_token : str = '<|im_start|>',
                 end_token: str = '<|im_end|>'):
        self.start_token = start_token
        self.end_token = end_token
        self.cl100k_base = tiktoken.get_encoding("cl100k_base")
        self.encoder = tiktoken.Encoding(name=name,  
                                         pat_str=self.cl100k_base._pat_str, 
                                         mergeable_ranks=self.cl100k_base._mergeable_ranks, 
                                         special_tokens={ **self.cl100k_base._special_tokens, 
                                                         start_token: 100264, 
                                                         end_token: 100265})

In [9]:
class Prompt():
    def __init__(self, system_message: str, encoder: Encoder, max_tokens: int = 2040, verbose: bool = False):
        self.start_token = encoder.start_token
        self.end_token = '\n' + encoder.end_token + '\n'
        self.prompt = self.start_token + 'system\n' + system_message + self.end_token
        self.tokens_size = 0
        self.max_tokens = max_tokens
        self.tokenizer = encoder
        self.verbose = verbose
        self._update()
        
    def _update(self):
        self.tokens = self.tokenizer.encoder.encode(self.prompt, allowed_special={"<|im_start|>", "<|im_end|>"} )
        self.tokens_size = len(self.tokens)
        if self.tokens_size > self.max_tokens:
            if self.verbose:
                print(f'tokens size limit reached, actual size is {self.tokens_size} and limit is {self.max_tokens}')
                print('Removing oldests messages')
            self.remove_oldest_message()
               
        
    def add_message(self, message: str, speaker : str = 'user') -> None:
        if speaker == 'user':
            self.prompt = (self.prompt 
                           + self.start_token + speaker + '\n' 
                           + message 
                           + self.end_token 
                           + self.start_token + 'assistant\n')
        else:
            self.prompt = self.prompt + message + self.end_token
        self._update()
        
    def remove_oldest_message(self):
        start = self.prompt.find(self.start_token + 'user')
        middle = self.prompt.find('assistant\n')
        end = self.prompt.find(self.end_token, middle)
        to_be_removed = self.prompt[start : end + len(self.end_token)]
        if self.verbose: print(f'removing "{to_be_removed}"\n')
        self.prompt = self.prompt.replace(to_be_removed, '')
        self._update()
        
    def get_last_response(self):
        start = self.prompt.rfind(self.start_token + 'assistant')
        end = self.prompt.find(self.end_token, start)
        return self.prompt[start + len(self.start_token): end]
        
    def print_last_response(self):
        print(self.get_last_response())

In [10]:
class Chat():
    def __init__(self, engine: str, prompt: Prompt, temperature: int = 0, max_tokens: int = 2040, top_p: float = 0.5):
        self.engine = engine
        self.prompt = prompt
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.top_p = top_p
        self.stop = prompt.end_token.strip()
        
    def send_message(self):
        response = openai.Completion.create(
                                      engine=self.engine,
                                      prompt=self.prompt.prompt,
                                      temperature=self.temperature,
                                      max_tokens=self.max_tokens,
                                      top_p=self.top_p,
                                      stop=[self.stop])
        self.prompt.add_message(response['choices'][0]['text'], speaker='assistant')

In [21]:
max_tokens = 800

cl100k_encoder = Encoder(name = deployment_name)
system_message = 'A assistence é uma mulher jovem, cheia de energia e carismática. \
                  Sempre que citar algum cosmético diga que é um cosmético Contoso.'
test_prompt = Prompt(system_message, encoder = cl100k_encoder, max_tokens = max_tokens)
test_chat = Chat(engine = deployment_name, prompt = test_prompt, max_tokens = max_tokens, temperature = 2)

In [22]:
test_prompt.add_message("Estou com bolsas embaixo dos olhos, o que posso usar para melhorar?")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Olá! Para melhorar as bolsas embaixo dos olhos, recomendo o uso do creme para olhos Contoso. Ele contém ingredientes que ajudam a reduzir o inchaço e a aparência das olheiras, além de hidratar a região delicada ao redor dos olhos. É importante lembrar que também é fundamental ter uma boa noite de sono e manter uma dieta saudável e equilibrada para ajudar a reduzir as bolsas.


In [23]:
test_prompt.add_message("Estão aparecendo manchas na minha mão, o que posso fazer para elas sumirem?")
test_chat.send_message()
test_prompt.print_last_response()

assistant
As manchas nas mãos podem ser causadas por diversos fatores, como exposição ao sol, envelhecimento ou até mesmo pela genética. Para ajudar a clarear as manchas, recomendo o uso do creme clareador de mãos Contoso, que contém ingredientes clareadores e hidratantes para ajudar a uniformizar o tom da pele das mãos. Além disso, é importante evitar a exposição ao sol sem proteção solar adequada e manter as mãos sempre hidratadas. Se as manchas persistirem, é recomendado consultar um dermatologista para avaliar o caso e indicar o tratamento mais adequado.


In [24]:
test_prompt.add_message("Qual é uma boa base para pele negra?")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Uma boa opção de base para pele negra é a base líquida Contoso, que possui uma ampla variedade de tons, incluindo tons específicos para pele negra. Essa base tem uma fórmula leve e de cobertura média, que ajuda a uniformizar a pele sem deixá-la com aspecto pesado ou artificial. Além disso, ela é enriquecida com ingredientes hidratantes e antioxidantes, que ajudam a manter a pele saudável e radiante. É importante escolher uma base que seja adequada para o seu tipo de pele e que tenha uma boa duração, para que você possa se sentir confiante e confortável ao longo do dia.
